# Proyecto 2

**Tema:** Predicción de enfermedades Cardíacas


|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |


## Medidas de Desempeño - Original

## Librerías Requeridas

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import KFold

## Carga de Datos

In [15]:
df = pd.read_csv("../../../Data/Transformed/Discrete.csv")

## Train and Test Set

In [16]:
variables = [column for column in list(df.columns) if column != 'heartdis']

X = df[variables].values
y = df['heartdis'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

df_train = pd.DataFrame(X_train, columns=variables)
df_train['heartdis'] = y_train

df_test = pd.DataFrame(X_test, columns=variables)
df_test['heartdis'] = y_test

## Train the Model

In [17]:
model = BayesianNetwork(
    [
        ('age', 'ca'),
        ('age', 'thalach'),
        ('age', 'trestbps'),
        ('sex', 'heartdis'),
        ('sex', 'thal'),
        ('slope', 'heartdis'),
        ('slope', 'oldpeak'),
        ('slope', 'thalach'),
        ('ca', 'heartdis'),
        ('exang', 'cp'),
        ('exang', 'oldpeak'),
        ('exang', 'thalach'),
        ('thal', 'exang'),
        ('thal', 'heartdis'),
        ('thal', 'oldpeak'),
        ('heartdis', 'cp'),
        ('heartdis', 'oldpeak'),
    ]
)

model.fit (
    data=df_train,
    estimator=MaximumLikelihoodEstimator,    
)
model.check_model()

nodes = model.nodes

## Test the Model

In [18]:
def inferenceEvidence(evidence):
    
    infer = VariableElimination(model)

    prob = infer.query(variables=['heartdis'], evidence=evidence)
    
    return prob.values.tolist()

In [19]:
def getEvidence(nodes, row):
    
    evidence = {}
    for node in nodes:
        if node != 'heartdis':
            evidence[node] = (row[node])
    
    return evidence

In [20]:
def getClassification(probs):
    cero = probs[0]
    one = probs[1]
    if cero >= one:
        result = 0
    else:
        result = 1
        
    return result

In [21]:
def getMetrics(df_test):
    
    # confussion_matrix = [[None, None],[None, None]]
    
    metrics = {'TN': None, 'FP': None, 'FN': None, 'TP': None}
    
    df_needed = df_test[['heartdis', 'test']]
    
    true_negatives_df = df_needed[(df_needed['heartdis'] == df_needed['test']) & (df_needed['test'] == 0)]
    true_negatives = true_negatives_df['test'].count()
    
    false_positives_df = df_needed[(df_needed['heartdis'] != df_needed['test']) & (df_needed['test'] == 1)]
    false_positives = false_positives_df['test'].count()
    
    false_negatives_df = df_needed[(df_needed['heartdis'] != df_needed['test']) & (df_needed['test'] == 0)]
    false_negatives = false_negatives_df['test'].count()
    
    
    true_positives_df = df_needed[(df_needed['heartdis'] == df_needed['test']) & (df_needed['test'] == 1)]
    true_positives = true_positives_df['test'].count()
    
    metrics['TN'] = true_negatives
    metrics['FP'] = false_positives
    metrics['FN'] = false_negatives
    metrics['TP'] = true_positives
    
    return metrics

In [22]:
test = []
for i in range(0, len(df_test)):
    row = df_test.iloc[i]
    evidence = getEvidence(nodes=nodes, row=row)
    probs = inferenceEvidence(evidence)
    classification = getClassification(probs)
    test.append(classification)

df_test['test'] = test
df_test

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:478: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdis,test
0,1,1,4,1,1,0,0,2,0,1,1,0,7,0,1
1,4,1,3,1,2,0,0,1,0,3,1,1,7,0,1
2,3,1,4,2,2,0,2,2,0,1,1,0,3,0,0
3,4,1,4,2,1,0,2,1,0,5,2,2,6,1,1
4,3,0,4,2,2,0,2,2,0,1,1,0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2,1,3,1,1,0,0,2,1,1,1,1,7,0,0
72,2,0,3,2,1,0,0,2,0,1,1,1,3,0,0
73,3,1,3,1,1,0,2,1,0,1,2,1,7,1,1
74,3,1,4,1,2,0,1,1,0,5,3,3,6,1,1


## Evaluation Metrics

In [25]:
metrics = getMetrics(df_test)

df_performance = pd.DataFrame([metrics])

df_performance['accuracy'] = (df_performance['TN'] + df_performance['TP']) / (df_performance['TN'] + df_performance['FP'] + df_performance['FN'] + df_performance['TP'])
df_performance['precision'] = (df_performance['TP']) / (df_performance['FP'] + df_performance['TP'])
df_performance['recall'] = (df_performance['TP']) / (df_performance['FN'] + df_performance['TP'])
df_performance['f1_score'] = 2 * (df_performance['precision'] * df_performance['recall']) / (df_performance['precision'] + df_performance['recall'])
df_performance['specificity'] = (df_performance['TN']) / (df_performance['TN'] + df_performance['FP'])

In [26]:
df_performance


,TN,FP,FN,TP,accuracy,precision,recall,f1_score,specificity
0,27,14,5,30,0.75,0.681818,0.857143,0.759494,0.658537


In [27]:
accuracy = df_performance['accuracy'].mean()
precision = df_performance['precision'].mean()
recall = df_performance['recall'].mean()
f1_score = df_performance['f1_score'].mean()
specificity = df_performance['specificity'].mean()

print(f'accuracy: \t {round(accuracy, 2)}')
print(f'precision: \t {round(precision, 2)}')
print(f'recall: \t {round(recall, 2)}')
print(f'f1_score: \t {round(f1_score, 2)}')
print(f'specificity: \t {round(specificity, 2)}')

accuracy: 	 0.75
precision: 	 0.68
recall: 	 0.86
f1_score: 	 0.76
specificity: 	 0.66
